# ANLP 2020 - Assignment 2

*Sophia Student, 1234567* (enter your name/student id number here)

<div class="alert alert-block alert-danger">Due: Wednesday, December 16, 2020, 10am</div>

<div class="alert alert-block alert-info">

**NOTE**
<br><br>

Please first fill in your name and id number at the top of the assignment, and **rename** the assignment file to **yourlastname-anlp-4.ipynb**<br><br>
Problems and questions are given in blue boxes like this one. All grey and white boxes must be filled by you (they either require code or a (brief!) discussion). <br><br>
Please hand in your assignment by the deadline via Moodle. In case of questions, you can contact the TAs or David via the usual channels.

</div>

<div class="alert alert-block alert-info">
    
In this assignment, you will implement a feedforward neural network and train it with backpropagation to classify intent from the provided dataset (<https://github.com/Dark-Sied/Intent_Classification>). For the purpose of understanding the learning process, the whole dataset is used as both training and test data. (What does that mean for your results?)<br><br>

You should implement all part of this exercise using only python + standard library + NumPy. (That is, no specialised machine learning libraries are allowed.) Here is a list of NumPy functions that may or may not be useful for this task: <br>
`np.array(), np.eye(), np.reshape(), np.ones(), np.zeros(), np.dot(), np.concatenate(), np.maximum(), np.argmax(), np.sum(), np.uniform()`. <br><br>

A more comprehensive introduction to NumPy can be found here: <https://sites.engineering.ucsb.edu/~shell/che210d/numpy.pdf> .

</div>

In [ ]:
# For your convenience, a function for reading in the dataset:
import csv

def load_dataset(filename):
    intent = []
    unique_intent = []
    sentences = []
    with open(filename, "r", encoding="latin1") as f:
        data = csv.reader(f, delimiter=",")
        for row in data:
            sentences.append(row[0])
            intent.append(row[1])
    unique_intent = set(intent)
    return sentences, intent, unique_intent
            
sentences, intent, unique_intent = load_dataset("dataset.csv")

## Problem 1: Bag-of-Words Representation [15pts]

<div class="alert alert-block alert-info">

The first thing you're being asked to do is to convert the text into a bag-of-words representation matrix where the dimension of the matrix is $V$ x $M$ ($M$: number of examples, $V$: vocabulary size) and the label to a matrix of dimension $K$ x $M$ where $K$ is number of classes.   

</div>

In [ ]:
# Student solution here.

## Problem 2: Activation Function [15 pts]

<div class="alert alert-block alert-info">
    
For the classification task, the softmax activation function for the output layer with K classes is given by: 
$softmax(z_i) = \frac{e^{z_i}}{{\sum_{j=1}^{K}e^{z_j}}}$ <br>
The activation function of the hidden neurons is a non-linear function. We have seen tanh being used in class, but more common these days are for example ReLU or sigmoid, given by: <br>
$ReLU(z)=max(0,z)$ <br>
$sigmoid(z)=\frac{1}{1+e^{-z}}$ <br>

Implement the softmax, ReLU, and sigmoid activation function in such a way that it accepts NumPy array and matrices. Plot the ReLU and sigmoid functions, as well as their derivatives. Observe the plot and discuss briefly what the advantages and disadvantages of the ReLU and sigmoid activation function might be. 

</div>

In [ ]:
# Student solution here.

Your discussion here

## Problem 3: Feedforward Neural Network [35 pts]

<div class="alert alert-block alert-info">
    
Now that you have created the input matrix, we can implement our neural network and perform a forward propagation to classify intent. To perform the forward propagation, you should compute $z^{l}$ and pass it through the activation function for each layer, given by: <br><br>
$z^{l} = W^{l}a^{l-1} + b^{l}$ <br>
$a^{l} = g(z^{l})$ <br>
where $W^{l}$ is a weight matrix between layer $l$ and $l+1$, $z^{l}$ is value of the hidden layer at layer $l$ before activation, $a^{l}$ is value of the hidden layer at layer $l$ after activation, and $b^{l}$ is bias term for layer $l$.

You should implement the feedforward computation that computes $\hat{y_{i}}$ for every example $i$. The neural network has 3 layers - an input layer, a hidden layer and an output layer, where the hidden layer has 150 neurons. Don't forget to include the bias term. Use ReLU as the activation function for the hidden layer and softmax for the output layer. For parameters initialization, use random values from uniform distribution in the range (-1,1). Provide a seed value to the random number generator, to make the results reproducible. The purpose of using this kind of initialisation is to break symmetry and ensure that different neurons can learn different non-linear functions. (Hint: use vectorization methods instead of a for loop for speedup.) <br><br>

Use this neural network to predict the intent and calculate the accuracy of the classifier. (Should you be expecting high numbers yet?)

</div>

In [ ]:
# Student solution here.

## Problem 4: Backpropagation [35 pts]

<div class="alert alert-block alert-info">
    
You will now implement the backpropagation algorithm to compute the gradient of the cost function with respect to the neural network weights' and bias term.  First of all, implement the cross entropy loss function to monitor if your model is actually learning. Remember that in backpropagation, we want to propagate the error signal to measure how much each neuron in the hidden layer contributes to the error in the output layer. It is more or less similar to forward propagation but in a reverse direction. For the output layer, set $\delta$ for cross entropy loss: <br><br>
$\delta^{L}= \hat{y} - y$ <br> where $L$ is the output layer and $\hat{y}$ is prediction of $y$. <br>

For the remaining hidden layer $l$, set: <br><br>
$\delta^{l} = (W^{l})^{T}\delta^{l+1} \odot g'(z^{l})$ <br> where $\odot$ is an element-wise product of matrices (Hadamard product), $g'$ is the derivative of the activation function. <br>

The derivative of the ReLU is given by:  $ReLU'(z) = \begin{cases} 1 & \text{if } z > 0 \\
                                                                                                                                      0 & \text{otherwise}.\end{cases}$<br>

By calculating the error term for each layer, you can then use the error term to calculate the partial derivatives $\frac{\partial \mathcal{L}}{\partial W^{l}} = \delta_{l+1} (a^{l})^{T}$ and $\frac{\partial \mathcal{L}}{\partial b^{l}} = \delta_{l+1}$ and perform batch gradient descent to update the parameter. (Batch gradient descent = run through all training instances and compute the gradient, then make the weight update.) Make sure that you accumulate the gradients for all the training samples and divide it by number of samples before doing the update. <br><br>

Here is some simple pseudocode to help with the training procedure: <br>
 * for number of epoch:
    > define gradient accumulator $\Delta w=0, \Delta b=0$ for each weight and bias term <br>
    > define cost accumulator $\Delta \mathcal{L}=0$ for the loss <br>
    
    > for each training example $i$:<br>
        >> perform forward propagation <br>
        >> calculate loss on example $i, L_{i}$ <br>
        >> $\Delta \mathcal{L} = \Delta \mathcal{L} + L_{i}$ <br> <br> 
        >> perform backpropagation <br>
        >> $\Delta w = \Delta w + \frac{\partial \mathcal{L}}{\partial W}$ for each weight <br>
        >> $\Delta b = \Delta b + \frac{\partial \mathcal{L}}{\partial b}$ for each bias term <br> 
        
    > calculate the cost, which is just average loss ($Cost = \frac{1}{m}\Delta \mathcal{L}$) <br>
    > $w = w - \frac{\alpha}{m}\Delta w$ for each weight <br>
    > $b = b - \frac{\alpha}{m}\Delta b$ for each bias term <br> 

Run the training for 1000 epoch using learning rate = 0.005 and use this neural network to predict the intent and calculate the accuracy of the classifier. (Hint: the dimension of $\delta^{l}$ should match the dimension of $a^{l}$, and the dimension of $\frac{\partial \mathcal{L}}{\partial W^{l}}$ and $\frac{\partial \mathcal{L}}{\partial b^{l}}$ should match the dimension of $W^{l}$ and $b^{l}$, respectively).<br><br>

Plot the cost function for each iteration and compare the results after training with results from Problem 3. Discuss what you observe!

</div>

In [ ]:
# Student solution here.

Your discussion here

## Bonus: Mini-Batch and Stochastic Gradient Descent [15pts]

<div class="alert alert-block alert-info">
    
As a bonus, train the neural network using mini-batch gradient descent with batch size = 64 and stochastic gradient descent (i.e., batch size = 1) for 1000 epoch using learning rate = 0.005. Plot the cost vs iteration for both cases and briefly discuss your observation!  

</div>

In [ ]:
# Student solution here.

Your discussion here